In [1]:
import sys
sys.path.append("../src")
import torch
import matplotlib.pyplot as plt
import numpy as np
import torchvision
import torch.nn.functional as F

import glob
import os
from datetime import datetime
import time
import math
from tqdm import tqdm

from itertools import repeat
from torch.nn.parameter import Parameter
import collections
import matplotlib
from torch_utils import *
from models import *
from visualization import *
# matplotlib.use('Agg')

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [3]:
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(), 
                                            torchvision.transforms.Normalize(mean=(0.0,), std=(1.0,))])

mnist_dset_train = torchvision.datasets.MNIST('./data', train=True, transform=transform, target_transform=None, download=True)
train_loader = torch.utils.data.DataLoader(mnist_dset_train, batch_size=20, shuffle=True, num_workers=0)

mnist_dset_test = torchvision.datasets.MNIST('./data', train=False, transform=transform, target_transform=None, download=True)
test_loader = torch.utils.data.DataLoader(mnist_dset_test, batch_size=20, shuffle=False, num_workers=0)

In [4]:
activation = hard_sigmoid
criterion = torch.nn.MSELoss(reduction='none').to(device)

In [5]:
architecture = [784, 500, 10]

x,y = next(iter(train_loader))
x = x.view(x.size(0),-1).to(device).T
y_one_hot = F.one_hot(y, 10).to(device).T

beta = 1
lambda_h = 0.9
lambda_y = 0.9
epsilon = 0.1
one_over_epsilon = 1 / epsilon
lr = {'ff' : 0.01, 'fb': 0.01, 'lat': 1e-3}
neural_lr = 0.001
model = TwoLayerCorInfoMaxHopfield(architecture = architecture, lambda_h = lambda_h, lambda_y = lambda_y, 
                           epsilon = epsilon, activation = activation)

In [5]:
trn_acc_list = []
tst_acc_list = []
neural_dynamic_iterations_free = 20
neural_dynamic_iterations_nudged = 4
# lambda_h = 0.01
# lambda_y = 0.01
# epsilon = 1
# one_over_epsilon = 1 / epsilon
n_epochs = 50
# lr = {'ff' : 1e-3, 'fb': 1e-3, 'lat': 1e-3}
# neural_lr = 0.25

for epoch_ in range(n_epochs):
    for idx, (x, y) in tqdm(enumerate(train_loader)):
        x, y = x.to(device), y.to(device)
        x = x.view(x.size(0),-1).T
        y_one_hot = F.one_hot(y, 10).to(device).T

        h, y_hat = model.batch_step(  x, y_one_hot, lr, neural_lr, neural_dynamic_iterations_free, 
                                      neural_dynamic_iterations_nudged, beta)
    
    trn_acc = evaluateCorInfoMax(model, train_loader, neural_lr, 20, device = 'cuda', printing = False)
    tst_acc = evaluateCorInfoMax(model, test_loader, neural_lr, 20, device = 'cuda', printing = False)
    trn_acc_list.append(trn_acc)
    tst_acc_list.append(tst_acc)
#     lr = {'ff' : 0.05 * (0.9)**epoch_, 'fb': 0.05 * (0.9) ** epoch_, 'lat': 1e-3}
    
    print("Epoch : {}, Train Accuracy : {}, Test Accuracy : {}".format(epoch_+1, trn_acc, tst_acc))

3000it [00:41, 72.74it/s]
7it [00:00, 66.03it/s]

Epoch : 1, Train Accuracy : 0.10178333333333334, Test Accuracy : 0.1046


248it [00:03, 72.07it/s]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/kuacc/users/bbozkurt15/.conda/envs/bbozkurt15/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_3912/868097122.py", line 19, in <module>
    h, y_hat = model.batch_step(  x, y_one_hot, lr, neural_lr, neural_dynamic_iterations_free,
  File "../src/models.py", line 162, in batch_step
    h, y_hat = self.run_neural_dynamics(x, h, y_hat, y_label, neural_lr,
  File "../src/models.py", line 132, in run_neural_dynamics
    grad_h, grad_y = self.calculate_neural_dynamics_grad(x, h, vh, y_hat, vy, y, beta)
  File "../src/models.py", line 118, in calculate_neural_dynamics_grad
    grad_h = -vh + 2* gam_h * M0 @ h - one_over_epsilon * (h - (Wff[0]['weight'] @ x + Wff[0]['bias'])) - one_over_epsilon * (h - (Wfb[1]['weight'] @ y_hat + Wfb[1]['bias']))
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceb

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/kuacc/users/bbozkurt15/.conda/envs/bbozkurt15/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_3912/868097122.py", line 19, in <module>
    h, y_hat = model.batch_step(  x, y_one_hot, lr, neural_lr, neural_dynamic_iterations_free,
  File "../src/models.py", line 162, in batch_step
    h, y_hat = self.run_neural_dynamics(x, h, y_hat, y_label, neural_lr,
  File "../src/models.py", line 132, in run_neural_dynamics
    grad_h, grad_y = self.calculate_neural_dynamics_grad(x, h, vh, y_hat, vy, y, beta)
  File "../src/models.py", line 118, in calculate_neural_dynamics_grad
    grad_h = -vh + 2* gam_h * M0 @ h - one_over_epsilon * (h - (Wff[0]['weight'] @ x + Wff[0]['bias'])) - one_over_epsilon * (h - (Wfb[1]['weight'] @ y_hat + Wfb[1]['bias']))
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceb


KeyboardInterrupt



In [233]:
architecture = [784, 500, 10]

x, y = next(iter(train_loader))

x, y = x.to(device), y.to(device)
x = x.view(x.size(0),-1).T
y_one_hot = F.one_hot(y, 10).to(device).T

beta = 1
lambda_h = 0.99
lambda_y = 0.99
epsilon = 0.01
one_over_epsilon = 1 / epsilon
lr = {'ff' : 0.01, 'fb': 0.01, 'lat': 1e-3}
model = TwoLayerCorInfoMaxHopfield(architecture = architecture, lambda_h = lambda_h, lambda_y = lambda_y, 
                           epsilon = epsilon, activation = activation, initialization = "xavier")

neural_lr = 0.001
h, y_hat = model.init_neurons(x.size(1), device = model.device, random_initialize = False)
neural_dynamic_iterations = 1
vh,vy = torch.clone(h), torch.clone(y_hat)
gam_h, gam_y = model.gam_h, model.gam_y
for iter_count in range(neural_dynamic_iterations):
    with torch.no_grad():       
        grad_h, grad_y = model.calculate_neural_dynamics_grad(x, h, vh, y_hat, vy, y, beta = 1)
        vh = vh + neural_lr * grad_h
        h = model.activation(0.5 * (1 / gam_h) * vh)

        vy = vy + neural_lr * grad_y
        y_hat = model.activation((1 / gam_y) * vy)

In [234]:
y_one_hot[:,1]

tensor([0, 0, 0, 0, 1, 0, 0, 0, 0, 0], device='cuda:0')

In [235]:
B, Wff, Wfb = model.B, model.Wff, model.Wfb

In [236]:
# h, y_hat = model.init_neurons(x.size(1), device = model.device, random_initialize = True)
# vh,vy = torch.clone(h), torch.clone(y_hat)
# M0 = B[0]['weight'] + torch.eye(B[0]['weight'].shape[0], requires_grad=False, device = model.device)
# M1 = B[1]['weight'] + torch.eye(B[1]['weight'].shape[0], requires_grad=False, device = model.device)
# -vy + gam_y * M1 @ y_hat - one_over_epsilon * (y_hat - (Wff[1]['weight'] @ h + Wff[1]['bias'])) + 2 * beta * (y - y_hat)

In [237]:
grad_y

tensor([[18.,  8.,  2., 12.,  2., 10., 10., 12.,  8.,  4., 18.,  0., 18., 10.,
          8.,  8., 16.,  6., 14., 14.],
        [18.,  8.,  2., 12.,  2., 10., 10., 12.,  8.,  4., 18.,  0., 18., 10.,
          8.,  8., 16.,  6., 14., 14.],
        [18.,  8.,  2., 12.,  2., 10., 10., 12.,  8.,  4., 18.,  0., 18., 10.,
          8.,  8., 16.,  6., 14., 14.],
        [18.,  8.,  2., 12.,  2., 10., 10., 12.,  8.,  4., 18.,  0., 18., 10.,
          8.,  8., 16.,  6., 14., 14.],
        [18.,  8.,  2., 12.,  2., 10., 10., 12.,  8.,  4., 18.,  0., 18., 10.,
          8.,  8., 16.,  6., 14., 14.],
        [18.,  8.,  2., 12.,  2., 10., 10., 12.,  8.,  4., 18.,  0., 18., 10.,
          8.,  8., 16.,  6., 14., 14.],
        [18.,  8.,  2., 12.,  2., 10., 10., 12.,  8.,  4., 18.,  0., 18., 10.,
          8.,  8., 16.,  6., 14., 14.],
        [18.,  8.,  2., 12.,  2., 10., 10., 12.,  8.,  4., 18.,  0., 18., 10.,
          8.,  8., 16.,  6., 14., 14.],
        [18.,  8.,  2., 12.,  2., 10., 10., 12.,

In [238]:
vy[:,1]

tensor([0.0080, 0.0080, 0.0080, 0.0080, 0.0080, 0.0080, 0.0080, 0.0080, 0.0080,
        0.0080], device='cuda:0')

In [239]:
y_hat[:,1]

tensor([0.7920, 0.7920, 0.7920, 0.7920, 0.7920, 0.7920, 0.7920, 0.7920, 0.7920,
        0.7920], device='cuda:0')

In [ ]:
architecture = [784, 500, 10]

x, y = next(iter(train_loader))

x, y = x.to(device), y.to(device)
x = x.view(x.size(0),-1).T
y_one_hot = F.one_hot(y, 10).to(device).T

beta = 1
lambda_h = 0.999
lambda_y = 0.999
epsilon = 0.1
one_over_epsilon = 1 / epsilon
lr = {'ff' : 0.01, 'fb': 0.01, 'lat': 1e-3}

model = TwoLayerCorInfoMax(architecture = architecture, lambda_h = lambda_h, lambda_y = lambda_y, 
                           epsilon = epsilon, activation = activation)

neural_lr = 0.0001
h, y_hat = model.init_neurons(x.size(1), device = model.device, random_initialize = False)
neural_dynamic_iterations = 4
for iter_count in range(neural_dynamic_iterations):
    with torch.no_grad():       
        grad_h, grad_y = model.calculate_neural_dynamics_grad(x, h, y_hat, y, beta = 1)
        h = model.activation(h + neural_lr * grad_h)
        y_hat = model.activation(y + neural_lr * grad_y)

In [ ]:
y_hat

In [ ]:
# vh = vh + neural_lr * grad_h
# h = model.activation(0.5 * (1 / model.gam_h) * vh)
neural_lr = 0.01
h, y_hat = model.init_neurons(x.size(1), device = model.device, random_initialize = False)
neural_dynamic_iterations = 4
vh,vy = torch.clone(h), torch.clone(y_hat)
gam_h, gam_y = model.gam_h, model.gam_y
# for iter_count in range(neural_dynamic_iterations):
#     with torch.no_grad():       
#         grad_h, grad_y = model.calculate_neural_dynamics_grad(x, h, vh, y_hat, vy, y, beta = 1)
#         vh = vh + neural_lr * grad_h
#         h = model.activation(0.5 * (1 / gam_h) * vh)

#         vy = vy + neural_lr * grad_y
#         y_hat = model.activation((1 / gam_y) * vy)
for iter_count in range(neural_dynamic_iterations):
    with torch.no_grad():       
        grad_h, grad_y = model.calculate_neural_dynamics_grad(x, h, vh, y_hat, vy, y, beta = 0)
        h = model.activation(h + neural_lr * grad_h)

        y_hat = model.activation(y + neural_lr * grad_y)

In [ ]:
torch.argmax(y_hat, 0)

In [ ]:
model.Wff

In [ ]:
y_hat

In [ ]:
(y_hat)